In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from scipy.stats import uniform, randint

In [ ]:
columns = ["Date", "Close", "High", "Low", "Open", "Volume"]

AAPL_prices = pd.read_csv("../data/raw/yf_prices/AAPL.csv")
AMZN_prices = pd.read_csv("../data/raw/yf_prices/AMZN.csv")
BA_prices = pd.read_csv("../data/raw/yf_prices/BA.csv")
CAT_prices = pd.read_csv("../data/raw/yf_prices/CAT.csv")
CVX_prices = pd.read_csv("../data/raw/yf_prices/CVX.csv")
GS_prices = pd.read_csv("../data/raw/yf_prices/GS.csv")
JNJ_prices = pd.read_csv("../data/raw/yf_prices/JNJ.csv")
JPM_prices = pd.read_csv("../data/raw/yf_prices/JPM.csv")
MSFT_prices = pd.read_csv("../data/raw/yf_prices/MSFT.csv")
NEE_prices = pd.read_csv("../data/raw/yf_prices/NEE.csv")
NVDA_prices = pd.read_csv("../data/raw/yf_prices/NVDA.csv")
PFE_prices = pd.read_csv("../data/raw/yf_prices/PFE.csv")
TSLA_prices = pd.read_csv("../data/raw/yf_prices/TSLA.csv")
XOM_prices = pd.read_csv("../data/raw/yf_prices/XOM.csv")

In [ ]:
def clean_prices_df(df):
    df = df.copy()
    df.rename(columns={"Price": "Date"}, inplace=True)
    df["Close"] = pd.to_numeric(df["Close"], errors="coerce")
    df = df.dropna(subset=["Close"]).reset_index(drop=True)
    for col in ["High", "Low", "Open", "Volume"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    return df

In [ ]:
AAPL_prices = clean_prices_df(AAPL_prices)
AMZN_prices = clean_prices_df(AMZN_prices)
BA_prices = clean_prices_df(BA_prices)
CAT_prices = clean_prices_df(CAT_prices)
CVX_prices = clean_prices_df(CVX_prices)
GS_prices = clean_prices_df(GS_prices)
JNJ_prices = clean_prices_df(JNJ_prices)
JPM_prices = clean_prices_df(JPM_prices)
MSFT_prices = clean_prices_df(MSFT_prices)
NEE_prices = clean_prices_df(NEE_prices)
NVDA_prices = clean_prices_df(NVDA_prices)
PFE_prices = clean_prices_df(PFE_prices)
TSLA_prices = clean_prices_df(TSLA_prices)
XOM_prices = clean_prices_df(XOM_prices)

In [ ]:
def feature_engineer_prices_df(df):
    df = df.copy()

    # --- Basic price change features ---
    df["Return_1d"] = df["Close"].pct_change()
    df["Return_7d"] = df["Close"].pct_change(7)

    # --- Moving averages ---
    df["MA_7"] = df["Close"].rolling(window=7).mean()
    df["MA_14"] = df["Close"].rolling(window=14).mean()
    df["MA_30"] = df["Close"].rolling(window=30).mean()

    # --- Volatility ---
    df["Volatility_7d"] = df["Return_1d"].rolling(window=7).std()
    df["Volatility_14d"] = df["Return_1d"].rolling(window=14).std()

    # --- Volume features ---
    df["Volume_MA_7"] = df["Volume"].rolling(window=7).mean()
    df["Volume_Change_1d"] = df["Volume"].pct_change()

    # --- Lag features (previous close prices) ---
    df["Close_Lag_1"] = df["Close"].shift(1)
    df["Close_Lag_2"] = df["Close"].shift(2)
    df["Close_Lag_3"] = df["Close"].shift(3)
    df["Close_Lag_5"] = df["Close"].shift(5)
    df["Close_Lag_7"] = df["Close"].shift(7)
    df["Close_Lag_14"] = df["Close"].shift(14)
    df["Close_Lag_21"] = df["Close"].shift(21)

    # --- RSI (Relative Strength Index) ---
    delta = df["Close"].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()

    rs = avg_gain / avg_loss
    df["RSI_14"] = 100 - (100 / (1 + rs))

    # --- MACD (Moving Average Convergence Divergence) ---
    # Fast and Slow Exponential Moving Averages
    df["EMA_12"] = df["Close"].ewm(span=12, adjust=False).mean()
    df["EMA_26"] = df["Close"].ewm(span=26, adjust=False).mean()

    # MACD Line
    df["MACD"] = df["EMA_12"] - df["EMA_26"]

    # Signal Line (9-day EMA of MACD)
    df["MACD_Signal"] = df["MACD"].ewm(span=9, adjust=False).mean()

    # interaction features
    df["Price_vs_MA30"] = df["Close"] / df["MA_30"]
    df["Vol_x_Return"] = df["Volume_Change_1d"] * df["Return_1d"]

    # Bollinger Bands
    df["MA_20"] = df["Close"].rolling(window=20).mean()
    df["Std_Dev_20"] = df["Close"].rolling(window=20).std()
    df["Bollinger_Upper"] = df["MA_20"] + (2 * df["Std_Dev_20"])
    df["Bollinger_Lower"] = df["MA_20"] - (2 * df["Std_Dev_20"])
    df["Price_vs_Bollinger"] = (df["Close"] - df["MA_20"]) / (
        2 * df["Std_Dev_20"]
    )  # Normalize position within bands

    # date features
    df["DayOfWeek"] = df["Date"].dt.dayofweek
    df["Month"] = df["Date"].dt.month
    df["WeekOfYear"] = df["Date"].dt.isocalendar().week.astype(int)
    df["Year"] = df["Date"].dt.year

    # range features
    df["Price_Range_HL"] = df["High"] - df["Low"]
    df["Price_Change_OC"] = df["Close"] - df["Open"]

    # --- Drop very early rows that have NaNs due to rolling windows ---
    df = df.dropna().reset_index(drop=True)

    return df

In [ ]:
AAPL_prices = feature_engineer_prices_df(AAPL_prices)
AMZN_prices = feature_engineer_prices_df(AMZN_prices)
BA_prices = feature_engineer_prices_df(BA_prices)
CAT_prices = feature_engineer_prices_df(CAT_prices)
CVX_prices = feature_engineer_prices_df(CVX_prices)
GS_prices = feature_engineer_prices_df(GS_prices)
JNJ_prices = feature_engineer_prices_df(JNJ_prices)
JPM_prices = feature_engineer_prices_df(JPM_prices)
MSFT_prices = feature_engineer_prices_df(MSFT_prices)
NEE_prices = feature_engineer_prices_df(NEE_prices)
NVDA_prices = feature_engineer_prices_df(NVDA_prices)
PFE_prices = feature_engineer_prices_df(PFE_prices)
TSLA_prices = feature_engineer_prices_df(TSLA_prices)
XOM_prices = feature_engineer_prices_df(XOM_prices)

In [ ]:
def prepare_data(df, target_col="Close", target_days=7):
    df = df.copy()

    # Keep the 'Close' price from the day the prediction is made FROM
    df["Prediction_Base_Price"] = df[target_col].shift(target_days)  # Price at t-7

    # Define the target as 7-day percentage return
    df["Target_Return"] = (
        df[target_col].pct_change(periods=target_days).shift(-target_days)
    )

    # Drop rows where target or base price is NaN
    # Also drop rows where original Close is NaN just in case
    df = df.dropna(subset=["Target_Return", "Prediction_Base_Price", target_col])

    # Store dates for plotting
    dates = df["Date"]

    # Define features (X) and target (y)
    # Ensure Prediction_Base_Price is NOT included in features unless intended
    features_to_drop = ["Target_Return", target_col, "Date", "Prediction_Base_Price"]
    features = df.drop(columns=[col for col in features_to_drop if col in df.columns])
    target = df["Target_Return"]
    base_prices = df["Prediction_Base_Price"]  # Keep this separate

    # Split data
    (
        X_train,
        X_test,
        y_train,
        y_test,
        dates_train,
        dates_test,
        base_prices_train,
        base_prices_test,
    ) = train_test_split(
        features, target, dates, base_prices, test_size=0.2, shuffle=False
    )

    # Return split data including base prices for evaluation
    # We only need to return the test versions of dates and base_prices usually
    return X_train, X_test, y_train, y_test, dates_test, base_prices_test

In [ ]:
def tune_and_train_xgboost(X_train, y_train, X_test, y_test):
    """
    Tunes XGBoost hyperparameters using RandomizedSearchCV with TimeSeriesSplit,
    retrains the best model with early stopping, and evaluates on the test set.

    Args:
        X_train (pd.DataFrame): Training features.
        y_train (pd.Series): Training target.
        X_test (pd.DataFrame): Testing features.
        y_test (pd.Series): Testing target.

    Returns:
        tuple: (best_model, y_pred)
            best_model (xgb.XGBRegressor): The final trained model.
            y_pred (np.array): Predictions on the test set.
    """

    # --- 1. Define Hyperparameter Search Space ---
    # Define ranges for hyperparameters you want to tune
    params = {
        "learning_rate": uniform(0.01, 0.15),  # e.g., values between 0.01 and 0.16
        "max_depth": randint(3, 8),  # e.g., integers from 3 up to 7
        "subsample": uniform(
            0.6, 0.4
        ),  # e.g., values between 0.6 and 1.0 (range size is 0.4)
        "colsample_bytree": uniform(0.6, 0.4),  # e.g., values between 0.6 and 1.0
        "gamma": uniform(0, 0.5),  # Regularization parameter
        "reg_alpha": uniform(0, 5),  # L1 regularization
        "reg_lambda": uniform(1, 5),  # L2 regularization (often good to have some L2)
        # n_estimators will be handled by early stopping later,
        # but we need a fixed value for the initial search phase
    }

    # --- 2. Setup Base Model (for search) & TimeSeriesSplit ---
    # Use a fixed, reasonable number of estimators for the search itself.
    # The search focuses on finding the best *other* parameters.
    search_xgb = xgb.XGBRegressor(
        objective="reg:squarederror",
        n_estimators=300,  # Fixed number for the search phase
        random_state=42,
        n_jobs=-1,
    )

    # Use TimeSeriesSplit for cross-validation
    # Adjust n_splits based on your data size and computational resources
    tscv = TimeSeriesSplit(n_splits=5)

    # --- 3. Setup and Run RandomizedSearchCV ---
    # n_iter controls how many parameter combinations are tried
    search = RandomizedSearchCV(
        estimator=search_xgb,
        param_distributions=params,
        n_iter=50,  # Increase/decrease based on time/resources (e.g., 25, 50, 100)
        scoring="neg_root_mean_squared_error",  # Optimize for RMSE (scikit-learn uses negative scores)
        cv=tscv,  # Use the time series cross-validator
        verbose=1,  # Set to 1 or 2 to see progress
        random_state=42,
        n_jobs=1,  # RandomizedSearch itself can be parallelized if needed, but XGBoost uses n_jobs already
        # Set to > 1 only if XGBoost n_jobs is 1 or if you have many CPU cores.
    )

    print("Starting hyperparameter search...")
    # Fit the search on the *entire* training dataset (X_train, y_train)
    search.fit(X_train, y_train)
    print("Search complete.")
    print(f"Best parameters found: {search.best_params_}")
    # Score is negative RMSE, so multiply by -1
    print(f"Best CV score (RMSE): {-search.best_score_:.4f}")

    # --- 4. Retrain the BEST Model with Early Stopping ---
    # Get the best hyperparameters found by the search
    best_params = search.best_params_

    # Create the final model configuration, adding n_estimators and early stopping
    final_model = xgb.XGBRegressor(
        objective="reg:squarederror",
        n_estimators=2000,  # Set a high number for early stopping to find the optimum
        early_stopping_rounds=50,  # Increase patience compared to your previous attempt
        random_state=42,
        n_jobs=-1,
        **best_params,  # Unpack the best parameters found (LR, depth, etc.)
    )

    # Create a final train/validation split *from the original training data*
    # This validation set is ONLY used for early stopping in this final training run
    X_train_final, X_val_final, y_train_final, y_val_final = train_test_split(
        X_train,
        y_train,
        test_size=0.2,
        shuffle=False,  # IMPORTANT: Keep temporal order
    )

    print("Training final model with best parameters and early stopping...")
    # Train the final model using early stopping
    final_model.fit(
        X_train_final,
        y_train_final,
        eval_set=[
            (X_val_final, y_val_final)
        ],  # Provide validation data for early stopping
        verbose=False,  # Set to True or e.g., 100 to see progress every 100 rounds
    )
    print(f"Best iteration found by early stopping: {final_model.best_iteration}")

    # --- 5. Evaluate on the Hold-Out Test Set ---
    # Predict on the original, unseen test set
    y_pred = final_model.predict(X_test)  # Predicts using the best iteration

    # Calculate final performance metric
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print(f"\nFinal Root Mean Squared Error (RMSE) on Test Set: {rmse:.4f}")

    return final_model, y_pred

In [ ]:
# Prepare data
X_train, X_test, y_train, y_test, dates_test, base_prices_test = prepare_data(
    AAPL_prices
)

# Train model
model, y_pred_returns = tune_and_train_xgboost(
    X_train, y_train, X_test, y_test
)  # Model predicts returns

predicted_prices = base_prices_test * (1 + y_pred_returns)
actual_prices = base_prices_test * (1 + y_test)

rmse_price = np.sqrt(mean_squared_error(actual_prices, predicted_prices))
print(f"\nFinal Price Root Mean Squared Error (RMSE) on Test Set: {rmse_price:.4f}")


# --- Plotting with actual dates ---
plt.figure(figsize=(12, 6))
plt.plot(dates_test, actual_prices, label="Actual Price", alpha=0.8)
plt.plot(dates_test, predicted_prices, label="Predicted Price", alpha=0.8)
plt.xlabel("Date")
plt.ylabel("Price")
plt.title("AAPL Price Prediction (7 Days Ahead)")  # Add title
plt.legend()
plt.xticks(rotation=45)  # Rotate date labels if needed
plt.tight_layout()  # Adjust layout
plt.show()

In [ ]:
# Prepare data
X_train, X_test, y_train, y_test, dates_test, base_prices_test = prepare_data(
    AMZN_prices
)

# Train model
model, y_pred_returns = tune_and_train_xgboost(
    X_train, y_train, X_test, y_test
)  # Model predicts returns

predicted_prices = base_prices_test * (1 + y_pred_returns)
actual_prices = base_prices_test * (1 + y_test)

rmse_price = np.sqrt(mean_squared_error(actual_prices, predicted_prices))
print(f"\nFinal Price Root Mean Squared Error (RMSE) on Test Set: {rmse_price:.4f}")


# --- Plotting with actual dates ---
plt.figure(figsize=(12, 6))
plt.plot(dates_test, actual_prices, label="Actual Price", alpha=0.8)
plt.plot(dates_test, predicted_prices, label="Predicted Price", alpha=0.8)
plt.xlabel("Date")
plt.ylabel("Price")
plt.title("AMZN Price Prediction (7 Days Ahead)")  # Add title
plt.legend()
plt.xticks(rotation=45)  # Rotate date labels if needed
plt.tight_layout()  # Adjust layout
plt.show()

In [ ]:
# Prepare data
X_train, X_test, y_train, y_test, dates_test, base_prices_test = prepare_data(
    BA_prices
)

# Train model
model, y_pred_returns = tune_and_train_xgboost(
    X_train, y_train, X_test, y_test
)  # Model predicts returns

predicted_prices = base_prices_test * (1 + y_pred_returns)
actual_prices = base_prices_test * (1 + y_test)

rmse_price = np.sqrt(mean_squared_error(actual_prices, predicted_prices))
print(f"\nFinal Price Root Mean Squared Error (RMSE) on Test Set: {rmse_price:.4f}")


# --- Plotting with actual dates ---
plt.figure(figsize=(12, 6))
plt.plot(dates_test, actual_prices, label="Actual Price", alpha=0.8)
plt.plot(dates_test, predicted_prices, label="Predicted Price", alpha=0.8)
plt.xlabel("Date")
plt.ylabel("Price")
plt.title("BA Price Prediction (7 Days Ahead)")  # Add title
plt.legend()
plt.xticks(rotation=45)  # Rotate date labels if needed
plt.tight_layout()  # Adjust layout
plt.show()

In [ ]:
# Prepare data
X_train, X_test, y_train, y_test, dates_test, base_prices_test = prepare_data(
    CAT_prices
)

# Train model
model, y_pred_returns = tune_and_train_xgboost(
    X_train, y_train, X_test, y_test
)  # Model predicts returns

predicted_prices = base_prices_test * (1 + y_pred_returns)
actual_prices = base_prices_test * (1 + y_test)

rmse_price = np.sqrt(mean_squared_error(actual_prices, predicted_prices))
print(f"\nFinal Price Root Mean Squared Error (RMSE) on Test Set: {rmse_price:.4f}")


# --- Plotting with actual dates ---
plt.figure(figsize=(12, 6))
plt.plot(dates_test, actual_prices, label="Actual Price", alpha=0.8)
plt.plot(dates_test, predicted_prices, label="Predicted Price", alpha=0.8)
plt.xlabel("Date")
plt.ylabel("Price")
plt.title("CAT Price Prediction (7 Days Ahead)")  # Add title
plt.legend()
plt.xticks(rotation=45)  # Rotate date labels if needed
plt.tight_layout()  # Adjust layout
plt.show()

In [ ]:
# Prepare data
X_train, X_test, y_train, y_test, dates_test, base_prices_test = prepare_data(
    CVX_prices
)

# Train model
model, y_pred_returns = tune_and_train_xgboost(
    X_train, y_train, X_test, y_test
)  # Model predicts returns

predicted_prices = base_prices_test * (1 + y_pred_returns)
actual_prices = base_prices_test * (1 + y_test)

rmse_price = np.sqrt(mean_squared_error(actual_prices, predicted_prices))
print(f"\nFinal Price Root Mean Squared Error (RMSE) on Test Set: {rmse_price:.4f}")


# --- Plotting with actual dates ---
plt.figure(figsize=(12, 6))
plt.plot(dates_test, actual_prices, label="Actual Price", alpha=0.8)
plt.plot(dates_test, predicted_prices, label="Predicted Price", alpha=0.8)
plt.xlabel("Date")
plt.ylabel("Price")
plt.title("CVX Price Prediction (7 Days Ahead)")  # Add title
plt.legend()
plt.xticks(rotation=45)  # Rotate date labels if needed
plt.tight_layout()  # Adjust layout
plt.show()

In [ ]:
# Prepare data
X_train, X_test, y_train, y_test, dates_test, base_prices_test = prepare_data(
    GS_prices
)

# Train model
model, y_pred_returns = tune_and_train_xgboost(
    X_train, y_train, X_test, y_test
)  # Model predicts returns

predicted_prices = base_prices_test * (1 + y_pred_returns)
actual_prices = base_prices_test * (1 + y_test)

rmse_price = np.sqrt(mean_squared_error(actual_prices, predicted_prices))
print(f"\nFinal Price Root Mean Squared Error (RMSE) on Test Set: {rmse_price:.4f}")


# --- Plotting with actual dates ---
plt.figure(figsize=(12, 6))
plt.plot(dates_test, actual_prices, label="Actual Price", alpha=0.8)
plt.plot(dates_test, predicted_prices, label="Predicted Price", alpha=0.8)
plt.xlabel("Date")
plt.ylabel("Price")
plt.title("GS Price Prediction (7 Days Ahead)")  # Add title
plt.legend()
plt.xticks(rotation=45)  # Rotate date labels if needed
plt.tight_layout()  # Adjust layout
plt.show()

In [ ]:
# Prepare data
X_train, X_test, y_train, y_test, dates_test, base_prices_test = prepare_data(
    JNJ_prices
)

# Train model
model, y_pred_returns = tune_and_train_xgboost(
    X_train, y_train, X_test, y_test
)  # Model predicts returns

predicted_prices = base_prices_test * (1 + y_pred_returns)
actual_prices = base_prices_test * (1 + y_test)

rmse_price = np.sqrt(mean_squared_error(actual_prices, predicted_prices))
print(f"\nFinal Price Root Mean Squared Error (RMSE) on Test Set: {rmse_price:.4f}")


# --- Plotting with actual dates ---
plt.figure(figsize=(12, 6))
plt.plot(dates_test, actual_prices, label="Actual Price", alpha=0.8)
plt.plot(dates_test, predicted_prices, label="Predicted Price", alpha=0.8)
plt.xlabel("Date")
plt.ylabel("Price")
plt.title("JNJ Price Prediction (7 Days Ahead)")  # Add title
plt.legend()
plt.xticks(rotation=45)  # Rotate date labels if needed
plt.tight_layout()  # Adjust layout
plt.show()

In [ ]:
# Prepare data
X_train, X_test, y_train, y_test, dates_test, base_prices_test = prepare_data(
    JPM_prices
)

# Train model
model, y_pred_returns = tune_and_train_xgboost(
    X_train, y_train, X_test, y_test
)  # Model predicts returns

predicted_prices = base_prices_test * (1 + y_pred_returns)
actual_prices = base_prices_test * (1 + y_test)

rmse_price = np.sqrt(mean_squared_error(actual_prices, predicted_prices))
print(f"\nFinal Price Root Mean Squared Error (RMSE) on Test Set: {rmse_price:.4f}")


# --- Plotting with actual dates ---
plt.figure(figsize=(12, 6))
plt.plot(dates_test, actual_prices, label="Actual Price", alpha=0.8)
plt.plot(dates_test, predicted_prices, label="Predicted Price", alpha=0.8)
plt.xlabel("Date")
plt.ylabel("Price")
plt.title("JPM Price Prediction (7 Days Ahead)")  # Add title
plt.legend()
plt.xticks(rotation=45)  # Rotate date labels if needed
plt.tight_layout()  # Adjust layout
plt.show()

In [ ]:
# Prepare data
X_train, X_test, y_train, y_test, dates_test, base_prices_test = prepare_data(
    MSFT_prices
)

# Train model
model, y_pred_returns = tune_and_train_xgboost(
    X_train, y_train, X_test, y_test
)  # Model predicts returns

predicted_prices = base_prices_test * (1 + y_pred_returns)
actual_prices = base_prices_test * (1 + y_test)

rmse_price = np.sqrt(mean_squared_error(actual_prices, predicted_prices))
print(f"\nFinal Price Root Mean Squared Error (RMSE) on Test Set: {rmse_price:.4f}")


# --- Plotting with actual dates ---
plt.figure(figsize=(12, 6))
plt.plot(dates_test, actual_prices, label="Actual Price", alpha=0.8)
plt.plot(dates_test, predicted_prices, label="Predicted Price", alpha=0.8)
plt.xlabel("Date")
plt.ylabel("Price")
plt.title("MSFT Price Prediction (7 Days Ahead)")  # Add title
plt.legend()
plt.xticks(rotation=45)  # Rotate date labels if needed
plt.tight_layout()  # Adjust layout
plt.show()

In [ ]:
# Prepare data
X_train, X_test, y_train, y_test, dates_test, base_prices_test = prepare_data(
    NEE_prices
)

# Train model
model, y_pred_returns = tune_and_train_xgboost(
    X_train, y_train, X_test, y_test
)  # Model predicts returns

predicted_prices = base_prices_test * (1 + y_pred_returns)
actual_prices = base_prices_test * (1 + y_test)

rmse_price = np.sqrt(mean_squared_error(actual_prices, predicted_prices))
print(f"\nFinal Price Root Mean Squared Error (RMSE) on Test Set: {rmse_price:.4f}")


# --- Plotting with actual dates ---
plt.figure(figsize=(12, 6))
plt.plot(dates_test, actual_prices, label="Actual Price", alpha=0.8)
plt.plot(dates_test, predicted_prices, label="Predicted Price", alpha=0.8)
plt.xlabel("Date")
plt.ylabel("Price")
plt.title("NEE Price Prediction (7 Days Ahead)")  # Add title
plt.legend()
plt.xticks(rotation=45)  # Rotate date labels if needed
plt.tight_layout()  # Adjust layout
plt.show()

In [ ]:
# Prepare data
X_train, X_test, y_train, y_test, dates_test, base_prices_test = prepare_data(
    NVDA_prices
)

# Train model
model, y_pred_returns = tune_and_train_xgboost(
    X_train, y_train, X_test, y_test
)  # Model predicts returns

predicted_prices = base_prices_test * (1 + y_pred_returns)
actual_prices = base_prices_test * (1 + y_test)

rmse_price = np.sqrt(mean_squared_error(actual_prices, predicted_prices))
print(f"\nFinal Price Root Mean Squared Error (RMSE) on Test Set: {rmse_price:.4f}")


# --- Plotting with actual dates ---
plt.figure(figsize=(12, 6))
plt.plot(dates_test, actual_prices, label="Actual Price", alpha=0.8)
plt.plot(dates_test, predicted_prices, label="Predicted Price", alpha=0.8)
plt.xlabel("Date")
plt.ylabel("Price")
plt.title("NVDA Price Prediction (7 Days Ahead)")  # Add title
plt.legend()
plt.xticks(rotation=45)  # Rotate date labels if needed
plt.tight_layout()  # Adjust layout
plt.show()

In [ ]:
# Prepare data
X_train, X_test, y_train, y_test, dates_test, base_prices_test = prepare_data(
    PFE_prices
)

# Train model
model, y_pred_returns = tune_and_train_xgboost(
    X_train, y_train, X_test, y_test
)  # Model predicts returns

predicted_prices = base_prices_test * (1 + y_pred_returns)
actual_prices = base_prices_test * (1 + y_test)

rmse_price = np.sqrt(mean_squared_error(actual_prices, predicted_prices))
print(f"\nFinal Price Root Mean Squared Error (RMSE) on Test Set: {rmse_price:.4f}")


# --- Plotting with actual dates ---
plt.figure(figsize=(12, 6))
plt.plot(dates_test, actual_prices, label="Actual Price", alpha=0.8)
plt.plot(dates_test, predicted_prices, label="Predicted Price", alpha=0.8)
plt.xlabel("Date")
plt.ylabel("Price")
plt.title("PFE Price Prediction (7 Days Ahead)")  # Add title
plt.legend()
plt.xticks(rotation=45)  # Rotate date labels if needed
plt.tight_layout()  # Adjust layout
plt.show()

In [ ]:
# Prepare data
X_train, X_test, y_train, y_test, dates_test, base_prices_test = prepare_data(
    TSLA_prices
)

# Train model
model, y_pred_returns = tune_and_train_xgboost(
    X_train, y_train, X_test, y_test
)  # Model predicts returns

predicted_prices = base_prices_test * (1 + y_pred_returns)
actual_prices = base_prices_test * (1 + y_test)

rmse_price = np.sqrt(mean_squared_error(actual_prices, predicted_prices))
print(f"\nFinal Price Root Mean Squared Error (RMSE) on Test Set: {rmse_price:.4f}")


# --- Plotting with actual dates ---
plt.figure(figsize=(12, 6))
plt.plot(dates_test, actual_prices, label="Actual Price", alpha=0.8)
plt.plot(dates_test, predicted_prices, label="Predicted Price", alpha=0.8)
plt.xlabel("Date")
plt.ylabel("Price")
plt.title("TSLA Price Prediction (7 Days Ahead)")  # Add title
plt.legend()
plt.xticks(rotation=45)  # Rotate date labels if needed
plt.tight_layout()  # Adjust layout
plt.show()